In [1]:
import autograd.numpy as agnp
import autograd as AG
import numpy as np
import tinympc

In [2]:
def hover_ref_func(num): #Returns Constant Hover Reference Trajectories At A Few Different Positions for Testing ([x,y,z,yaw])
    """ Returns constant hover reference trajectories at a few different positions. """
    hover_dict = {
        1: np.array([[0.0, 0.0, -0.6,     0.0, 0.0, 0.0,   0.0, 0.0, 0.0]]).T,
        2: np.array([[0.0, 0.8, -0.8,     0.0, 0.0, 0.0,   0.0, 0.0, 0.0]]).T,
        3: np.array([[0.8, 0.0, -0.8,     0.0, 0.0, 0.0,   0.0, 0.0, 0.0]]).T,
        4: np.array([[0.8, 0.8, -0.8,     0.0, 0.0, 0.0,   0.0, 0.0, 0.0]]).T,
        5: np.array([[0.0, 0.0, -10.0,    0.0, 0.0, 0.0,   0.0, 0.0, 0.0]]).T,
        6: np.array([[1.0, 1.0, -4.0,     0.0, 0.0, 0.0,   0.0, 0.0, 0.0]]).T,
        7: np.array([[3.0, 4.0, -5.0,     0.0, 0.0, 0.0,   0.0, 0.0, 0.0]]).T,
        8: np.array([[1.0, 1.0, -3.0,     0.0, 0.0, 0.0,   0.0, 0.0, 0.0]]).T,
    }
        
    print(f"hover_dict #{num}")
    r = hover_dict.get(num)
    r_final = np.tile(r, (1, N))
    return r_final

For quad

In [3]:
# Parameters
m = 2.1 # mass of the quad (kg)
g = 9.806 # gravity (m/s^2)


In [4]:
# Dynamics function
def quad_dynamics(X, U):
    """Quadrotor Dynamics Function."""
    x,y,z,vx,vy,vz,roll,pitch,yaw = X
    thrust, rolldot, pitchdot, yawdot = U

    sr = agnp.sin(roll)
    sy = agnp.sin(yaw)
    sp = agnp.sin(pitch)
    cr = agnp.cos(roll)
    cp = agnp.cos(pitch)
    cy = agnp.cos(yaw)

    #define dynamics
    pxdot = vx
    pydot = vy
    pzdot = vz
    vxdot = -(thrust/m) * (sr*sy + cr*cy*sp)
    vydot = -(thrust/m) * (cr*sy*sp - cy*sr)
    vzdot = g - (thrust/m) * (cr*cp)
    rolldot = rolldot
    pitchdot = pitchdot
    yawdot = yawdot

    return agnp.array([pxdot, pydot, pzdot, vxdot, vydot, vzdot, rolldot, pitchdot, yawdot])

# Runge-Kutta 4th order integration function
def quad_rk4(x, u, dt=0.01):
    f1 = dt * quad_dynamics(x, u)
    f2 = dt * quad_dynamics(x + f1 / 2, u)
    f3 = dt * quad_dynamics(x + f2 / 2, u)
    
    f4 = dt * quad_dynamics(x + f3, u)
    return x + (1 / 6) * (f1 + 2 * f2 + 2 * f3 + f4)

In [5]:
xgoal = agnp.array([0.0834213, 0.0323, -0.24,   0.00121, 0.0039234, 0.000324,   0.000432, 0.004032, 0.00043242])
ugoal = agnp.array([-m*g, 0.335, 0.5435, 0.35])

# Compute Jacobians
AGA = AG.jacobian(lambda x_: quad_rk4(x_, ugoal))(xgoal)
AGB = AG.jacobian(lambda u_: quad_rk4(xgoal, u_))(ugoal)
print(f"{AGA.shape= }")
print(f"{AGB.shape= }")

AGA.shape= (9, 9)
AGB.shape= (9, 4)


In [6]:
Q = np.diag([1000.0, 1000.0, 1000.0, 10., 10., 10., 10., 10., 100.0]);
R = np.diag([100.0]*4); # 

In [7]:
max_rate = 0.8
min_thrust = 0.0
max_thrust = 27.0
u_min = np.array([min_thrust, -max_rate, -max_rate, -max_rate])
u_max = np.array([max_thrust, max_rate, max_rate, max_rate])

In [8]:
N = 20
# Set up the problem
prob = tinympc.TinyMPC()
prob.setup(AGA, AGB, Q, R, N)

In [9]:
# Define initial condition
x0 = np.array([0.5, 1.3, -0.7,   0, 0, 0,   0, 0, 0])
reffunc = hover_ref_func(8)

hover_dict #8


In [10]:
# Set the first state in the horizon
prob.set_x0(x0)
prob.set_x_ref(reffunc) # Set the goal position

In [11]:
# Solve the problem
solution = prob.solve()

# Print the controls at the first time step
print(solution["controls"])

[ 7.18601890e+00  9.33580703e-01  1.57120459e+00 -2.94656072e-03]


In [12]:
solution['controls'].shape

(4,)

In [83]:
solution['states_all'].shape

(20, 9)

Compile in C

In [12]:
prob.codegen("generated_code", verbose=1)


Data header generated in /home/factslabegmc/final_wardi_files/src/quad_mpc/quad_mpc/generated_code//tinympc/tiny_data.hpp
Data generated in /home/factslabegmc/final_wardi_files/src/quad_mpc/quad_mpc/generated_code//src/tiny_data.cpp
Example tinympc main generated in /home/factslabegmc/final_wardi_files/src/quad_mpc/quad_mpc/generated_code//src/tiny_main.cpp
running build_ext
-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- pybind11 v2.14.0 dev1
-- Found PythonInterp: /home/factslabegmc/miniconda3/envs/wardiNN/bin/python (found suitable v

In [18]:
import numpy as np
import tinympcgen
import matplotlib.pyplot as plt

tinympcgen.set_x_ref(reffunc[:,-1].flatten()) # Set the goal position 
tinympcgen.set_x0(x0) # Set first state in horizon
solution = tinympcgen.solve()
print(solution["controls"])

[ 7.18601890e+00  9.33580703e-01  1.57120459e+00 -2.94656072e-03]


In [16]:
tinympcgen.

array([ 1.,  1., -3.,  0.,  0.,  0.,  0.,  0.,  0.])